In particular, in the context of languages with first-class functions, Norvig suggests rethinking the Strategy, Command, Template Method, and Visitor patterns. The general idea is: you can replace instances of some participant class in these patterns with simple functions, reducing a lot of boilerplate code. In this chapter, we will refactor Strategy using function objects, and discuss a similar approach to simplifying the Command pattern.

### Case Study: Refactoring Strategy

Strategy is a good example of a design pattern that can be simpler in Python if you leverage functions as first-class objects.

In the following section, we describe and implement Strategy using the “classic” structure described in Design Patterns.

#### Classic Strategy

The UML class diagram in Figure depicts an arrangement of classes that exemplifies the Strategy pattern.

![](images/Capture11.JPG)

UML class diagram for order discount processing implemented with the Strategy design pattern

The Strategy pattern is summarized like this in Design Patterns:
    
    Define a family of algorithms, encapsulate each one, and make them interchangeable. Strategy lets the algorithm vary independently from clients that use it.

Consider an online store with these discount rules:
    - Customers with 1,000 or more fidelity points get a global 5% discount per order.
    - A 10% discount is applied to each line item with 20 or more units in the same order.
    - Orders with at least 10 distinct items get a 7% global discount.

In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # the Strategy: an abstract base class

    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

Note that in Example, I coded Promotion as an abstract base class (ABC), to be able to use the @abstractmethod decorator, thus making the pattern more explicit.

![](images/Capture12.JPG)

In [2]:
dir(Order)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'due',
 'total']

Sample Usage of Order class with different promotions applied:

In [4]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [6]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [7]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [8]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [9]:
banana_cart = [LineItem('banana', 30, .5),
                LineItem('apple', 10, 1.5)]

In [10]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [11]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [12]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [13]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

Example works perfectly well, but the same functionality can be implemented with less code in Python by using functions as objects. The next section shows how.

### Function-Oriented Strategy

- Each concrete strategy in Example is a class with a single method, discount.
- You could say they look a lot like plain functions, and you would be right.

Order class with discount strategies implemented as functions

In [16]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

- To compute a discount, just call the self.promotion() function. 
- No abstract class. 
- Each strategy is a function. 

Sample usage of Order class with promotions as functions

In [19]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

Order(joe, cart, fidelity_promo)   

<Order total: 42.00 due: 42.00>

- To apply a discount strategy to an Order, just pass the promotion function as an argument.

Note the callouts in Example: there is no need to instantiate a new promotion object with each new order: the functions are ready to use.

By all means use plain old functions instead of coding single-method classes implementing a single-method interface declared in yet another class. A function is more lightweight than an instance of a user-defined class.

### Choosing the Best Strategy: Simple Approach

best_promo finds the maximum discount iterating over a list of functions

In [22]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

In [23]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

Although Example works and is easy to read, there is some duplication that could lead to a subtle bug: 
- to add a new promotion strategy, we need to code the function and remember to add it to the promos list, or else the new promotion will work when explicitly passed as an argument to Order, but will not be considered by best_promotion.

### Finding Strategies in a Module

Modules in Python are also first-class objects, and the standard library provides several functions to handle them. 

globals()
    
    Return a dictionary representing the current global symbol table. This is always the dictionary of the current module

The promos list is built by introspection of the module global namespace

In [24]:
promos = [globals()[name] for name in globals()
            if name.endswith('_promo')
            and name != 'best_promo']

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)   

-     Iterate over each name in the dictionary returned by globals(). 
-     Select only names that end with the _promo suffix. 

The promos list is built by introspection of a new promotions module

In [ ]:
promos = [func for name, func in
                inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

The function inspect.getmembers returns the attributes of an object—in this case, the promotions module—optionally filtered by a predicate (a boolean function). We use inspect.isfunction to get only the functions from the module.

A more explicit alternative for dynamically collecting promotional discount functions would be to use a simple decorator. 

### Command

Command is another design pattern that can be simplified by the use of functions passed as arguments.

![](images/Capture13.JPG)

The goal of Command is to decouple an object that invokes an operation (the Invoker) from the provider object that implements it (the Receiver).

- Instead of giving the Invoker a Command instance, we can simply give it a function. 
- Instead of calling command.execute(), the Invoker can just call command(). 
- The MacroCommand can be implemented with a class implementing __call__. Instances of MacroCommand would be callables, each holding a list of functions for future invocation, as implemented in Example.

In [41]:
class MacroCommand:
    """A command that executes a list of commands"""

    def __init__(self, commands):
        self.commands = list(commands)  # 1

    def __call__(self):
        for command in self.commands:  # 2
            command()

In [42]:
x = MacroCommand(list())

- A callable instance like MacroCommand in Example 6-9 can keep whatever state is necessary, and provide extra methods in addition to \_\_call\_\_.
- A closure can be used to hold the internal state of a function between calls. 


This concludes our rethinking of the Command pattern with first-class functions. At a high level, the approach here was similar to the one we applied to Strategy: replacing with callables the instances of a participant class that implemented a single-method interface. After all, every Python callable implements a single-method interface, and that method is named \_\_call\_\_.